In [1]:
import os
import pandas as pd
from sklearn.externals import joblib
import tensorflow as tf
import numpy as np

#피쳐로 추출한 테스트데이터 불러오기
#with_suspicious_perm.csv : 악성코드에서 자주 사용되는 권한만 사용
#악성앱과 정상앱 각각 불러오기
normal_path='./extract-from-apk-master/out/0-normal/with_suspicious_perm.csv'
malware_path='./extract-from-apk-master/out/1-malware/with_suspicious_perm.csv'
test_path='./extract-from-apk-master/out/test/with_suspicious_perm.csv'

#csv로 읽기
normal_data=pd.read_csv(normal_path)
malware_data=pd.read_csv(malware_path)
test_data=pd.read_csv(test_path)

#데이터 구조 확인
#normal_data : (4000, 141)
#malware_data : (1988,141)  2개는 dex파일 에러로 피쳐추출 실패
print(normal_data.shape, malware_data.shape, test_data.shape)

(5261, 141) (2737, 141) (1978, 141)


In [2]:
#악성, 정상앱 학습데이터 병합
data=pd.concat([normal_data, malware_data])

#데이터 구조 확인
#(5998,141)
print(data.shape)

(7998, 141)


In [3]:
#의미없는 피쳐 제거
test_data=test_data.iloc[:,(data.max()!=data.min()).tolist()[:-1]]

#데이터 구조 확인
#(5998, 112)
print(test_data.shape)

(1978, 115)


In [4]:
test_data['size']=(test_data['size']-np.mean(data['size']))/np.std(data['size'])
test_data['ep']=(test_data['ep']-np.mean(data['ep']))/np.std(data['ep'])

In [5]:
from sklearn.externals import joblib
def check(x_test):
    
    model_XGB=joblib.load('./XGB_model.pkl')
    y_pred_XGB = model_XGB.predict(x_test)
    
    model_RF=joblib.load('./RF_model.pkl')
    y_pred_RF = model_RF.predict(x_test)
    
    model_SVM=joblib.load('./SVM_model.pkl')
    y_pred_SVM = model_SVM.predict(x_test)
    
    
    #parameters
    n_inputs=114 #suspiciou = 135 full=229
    n_hidden1=200
    n_hidden2=200
    n_outputs=2

    scale1=0.001
    scale2=0.001

    dropout_rate=0.1 # == 1 - keep_prob

    learning_rate=0.03

    n_epochs = 1000
    display_step=10
    batch_size = 128
    batch_num=5998//batch_size

    tf.reset_default_graph()

    X=tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
    Y=tf.placeholder(tf.int64, shape=(None), name="Y")

    training=tf.placeholder_with_default(False, shape=(), name='training')

    with tf.name_scope("dnn"):
        X_drop=tf.layers.dropout(X, dropout_rate, training=training)
        hidden1 = tf.layers.dense(X_drop, n_hidden1, name="hidden1", activation=tf.nn.relu, kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale1, scale2))
        hidden1_dropp=tf.layers.dropout(hidden1, dropout_rate, training=training)
        hidden2 = tf.layers.dense(hidden1_dropp, n_hidden2, name="hidden2", activation=tf.nn.relu, kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale1, scale2))
        hidden2_dropp=tf.layers.dropout(hidden2, dropout_rate, training=training)
        logits = tf.layers.dense(hidden2_dropp, n_outputs, name="outputs")
    
    with tf.name_scope("loss"):
        xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
        loss=tf.reduce_mean(xentropy, name="loss")
        loss_summary = tf.summary.scalar('loss',loss)
    
    with tf.name_scope("train"):
        optimizer=tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
    
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, Y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        accuracy_summary = tf.summary.scalar('accuracy',accuracy)
    
    merged = tf.summary.merge_all()

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, "./mymodel_final.ckpt")
        Z=logits.eval(feed_dict={X:x_test})
        y_pred_DNN=np.argmax(Z, axis=1)
        
        
    df=pd.DataFrame([y_pred_DNN,y_pred_RF,y_pred_SVM,y_pred_XGB], index=['DNN', 'RF', 'SVM', 'XGB'])
    df=df.transpose()
    df['EN']=df.apply(lambda x: 1 if (x['DNN']+x['RF']+x['SVM']+x['XGB'])/4>=0.5 else 0, axis=1)
    y_pred_EN=df['EN']
    
    return y_pred_EN

In [6]:
test_data['pred']=check(test_data.iloc[:,1:])

INFO:tensorflow:Restoring parameters from ./mymodel_final.ckpt


In [7]:
test_data.iloc[:,[0,-1]].to_csv('out.csv')